In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge
from keras.preprocessing.text import one_hot
from keras.preprocessing import sequence
import numpy as np
from matplotlib import cm
import math
from keras.models import Sequential,Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, MaxoutDense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge,Input,merge,UpSampling2D
from matplotlib.patches import Circle
from os import listdir
from os.path import isfile, join
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import re
from numpy.random import random, permutation, randn, normal 
import PIL.Image
import os
import json
from tqdm import tqdm
from multiprocessing import Pool


from keras.optimizers import SGD, RMSprop, Adam

import cPickle as pickle
from matplotlib import pyplot as plt

from itertools import compress

import shutil
import string

import collections

import bcolz
import cPickle as pickle

from sklearn.utils import shuffle

import cv2

from skimage.transform import (hough_line, hough_line_peaks,
                               probabilistic_hough_line)

from IPython.display import Image, display, SVG
from keras.utils.visualize_util import model_to_dot

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
def get_nr_of_examples(df_path):
    
    split_paths = [split_path for split_path in os.listdir(df_path)]
    split_paths.sort()
    
    nr_examples = 0
    
    for split_path in tqdm(split_paths):

        data_df_split = pd.read_pickle(df_path + split_path)
        
        nr_examples += len(data_df_split)
    
    return nr_examples

def create_folder(path):
    if not os.path.isdir(path):
        os.mkdir(path)
    
def auto_canny(image, sigma=0.33):
    v = np.median(image)

    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    
    edged = cv2.Canny(image, lower, upper)

    return edged


def read_roma_paths(folders):
    
    all_img_paths = []
    all_mask_paths = []

    for folder in folders:
        img_paths = sorted([folder+path for path in os.listdir(folder) if ".jpg" in path])
        mask_paths = sorted([folder+path for path in os.listdir(folder) if ".pgm" in path])

        all_img_paths += img_paths
        all_mask_paths += mask_paths
        
    return all_img_paths, all_mask_paths

def arr(img):
    return np.asarray(img)
    


def crop_resize_data(imgs,masks,new_img_size):
    
    cropped_imgs = []
    cropped_masks = []
    
    for img,mask in tqdm(zip(imgs,masks)):
        cropped_img, cropped_mask = get_vp_img_based_on_haugh(img,mask)
        
        cropped_img = cropped_img.resize((new_img_size,new_img_size), PIL.Image.NEAREST)
        cropped_mask = cropped_mask.resize((new_img_size,new_img_size), PIL.Image.NEAREST)
        
        if(len(arr(cropped_mask).shape) == 3):
            cropped_mask = arr(cropped_mask)[...,0]
        
        cropped_imgs.append(arr(cropped_img))
        cropped_masks.append(np.expand_dims(arr(cropped_mask),axis=3))
        
    return np.stack(cropped_imgs),np.stack(cropped_masks)

def plot_predicted_vs_ground_truth(images, masks, predictions, nr_imgs = None):
        
    if(nr_imgs == None):
        nr_imgs = len(images)
        
    for index in tqdm(range(nr_imgs)):
   
        raw_pred = predictions[index]
        squeezed_pred = np.squeeze(raw_pred)

        raw_mask = masks[index]
        raw_mask = np.squeeze(raw_mask)
        
        image = images[index]
    
        nr_cols = 3
        
        plt.subplot(1,nr_cols,1)
        plt.imshow(image)
        
        plt.subplot(1,nr_cols,2)
        plt.imshow(raw_mask)
        
        plt.subplot(1,nr_cols,3)
        plt.imshow(squeezed_pred)
       
        plt.figure(figsize=(10,10))
        
    plt.show()

    
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0]*p2[1] - p2[0]*p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y

    return False
  

def get_vp_img_based_on_haugh(original_image,original_mask):
    
    original_image_width, original_image_height = original_image.size
        
    resized_img = original_image.resize((448,448), PIL.Image.NEAREST)
    resized_img_width, resized_img_height = resized_img.size
    
    resized_img = np.asarray(resized_img)
    canny_img = auto_canny(resized_img)
        
    h, theta, d = hough_line(canny_img)
    
    peaks = None
    peaks = zip(*hough_line_peaks(h, theta, d, threshold=25))
    nr_peaks = 5
    

    angle_2_dist_list = []
    
    for _, angle, dist in peaks[:nr_peaks]:
        if(angle < 1.5 and angle > -1.5):
            angle_2_dist_list.append((angle,dist))
    
    if(len(angle_2_dist_list) < 2):
        return original_image,original_mask
    
        
    angle_2_dist_list = sorted(angle_2_dist_list, key=lambda tup: tup[0])

    selected_angle_dist_list = [angle_2_dist_list[0],angle_2_dist_list[len(angle_2_dist_list)-1]]

    
    lines = []

    for angle,dist in selected_angle_dist_list:
        y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
        y1 = (dist - canny_img.shape[1] * np.cos(angle)) / np.sin(angle)
        
        lines.append(([0,y0],[canny_img.shape[1],y1]))
      
    L1 = line(lines[0][0],lines[0][1])
    L2 = line(lines[1][0],lines[1][1])
    
    R = intersection(L1, L2)
    
    if (not R):
        return original_image,original_mask
    
    new_width = max(0,int(R[0]))
    new_width = min(resized_img_width,new_width)
    
    new_height = max(0,int(R[1]))
    new_height = min(resized_img_height,new_height)

    height_percentage = float(new_height) / resized_img_height
    vp_width_percentage = float(new_width) / resized_img_width
    
    if(height_percentage == 1):
        return original_image,original_mask
    
    original_img_new_height = int(original_image_height * height_percentage)
    
    cropped_img = original_image.crop((0,original_img_new_height,original_image_width,original_image_height))
    cropped_mask = original_mask.crop((0,original_img_new_height,original_image_width,original_image_height))
    
    return cropped_img,cropped_mask   


def rotate_image(image,angle):
    height, width, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
    result = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR)
    return result
    
def rotate_image_cw(img):
    return rotate_image(img,-15)

def rotate_image_ccw(img):
    return rotate_image(img,15)

def flip_image(img):
    return cv2.flip(img,1)

def read_camvid_mask(path):
    
    mask = arr(PIL.Image.open(path).resize((480,360))).copy()
    
    idx = (mask[...,0] == 128) & (mask[...,1] == 0) & (mask[...,2] == 192)
    not_idx = np.logical_not(idx)

    mask[idx] = 1
    mask[not_idx] = 0
    
    return PIL.Image.fromarray(mask)


def normalize_imgs(data):
    
    data = data.astype('float32')

    mean = np.mean(data)  # mean for data centering
    std = np.std(data)  # std for data normalization

    data -= mean
    data /= std
    
    return data

def transform_camvid_masks(camvid_mask_paths):
    
    for path in tqdm(camvid_mask_paths):
        img_name = os.path.basename(path)
        marking_mask = read_camvid_mask(path)
        output_path = "../datasets/camvid/rm_masks/"+img_name
        marking_mask.save(output_path)
        
    
def get_roma_img_mask_paths(roma_dataset_path):
    roma_folders = [roma_dataset_path + folder+"/" for folder in os.listdir(roma_dataset_path)]
    img_paths, mask_paths = read_roma_paths(roma_folders)
    
    data_df = pd.DataFrame({
        'img_path': img_paths,
        'img_name': [os.path.basename(path) for path in img_paths],
        'mask_path': mask_paths
    })
    
    data_df = data_df.set_index('img_name')
    
    return data_df

def get_camvid_img_mask_paths(camvid_dataset_path):

    camvid_img_path = camvid_dataset_path + "imgs/"
    camvid_mask_path = camvid_dataset_path + "rm_masks/"
    
    img_paths = [camvid_img_path + path for path in sorted(os.listdir(camvid_img_path))]
    mask_paths = [camvid_mask_path + path for path in sorted(os.listdir(camvid_mask_path))]
  
    data_df = pd.DataFrame({
        'img_path': img_paths,
        'img_name': [os.path.basename(path) for path in img_paths],
        'mask_path': mask_paths
    })
    
    data_df = data_df.set_index('img_name',)
    
    return data_df


def get_data_from_paths(img_paths, mask_paths, nr_imgs = None):
      
    if(nr_imgs == None):
        nr_imgs = len(img_paths)
    
 
    masks = [PIL.Image.open(path) for path in tqdm(mask_paths[:nr_imgs])]
    
    imgs = shuffle(imgs, random_state=0)
    masks = shuffle(masks, random_state=0)

    return imgs,masks

def plot_img_mask(imgs,masks,nr_to_plot = None):
       
    if(nr_to_plot == None):
        nr_imgs = len(imgs)
   
    for i in range(10):
    
        img = imgs[i]
        mask = np.squeeze(masks[i])

        plt.subplot(1,2,1)
        plt.imshow(img)

        plt.subplot(1,2,2)
        plt.imshow(mask)

        plt.figure()

        plt.show()
        
        
def numpify(series):  
    return np.stack(series.tolist())

def split_train_test(data_df, train_percentage):
    
    nr_train = int(train_percentage * len(data_df))
    train_df = data_df[:nr_train]
    test_df = data_df[nr_train:]
    
    return train_df, test_df
    
def read_data(img_path, mask_path, new_img_size):
    img =  PIL.Image.open(img_path).resize((new_img_size,new_img_size))
    mask = PIL.Image.open(mask_path).resize((new_img_size,new_img_size))
    
    #mask = np.squeeze(mask).copy()
        
    if(len(arr(mask).shape) != 2):
        mask = arr(mask)[...,0]
        
    mask = arr(mask) / 255.
    
    mask = PIL.Image.fromarray(mask)
    
    return pd.Series({'img': img, \
                      'mask': mask})

def construct_batch((data_df_split, df_path, img_path, mask_path, index, new_img_size)):
        
        
    data_df_split = pd.concat([data_df_split, data_df_split.apply(lambda row : \
                                                                  read_data(row['img_path'],\
                                                                             row['mask_path'],\
                                                                             new_img_size),axis=1)],axis=1)

    imgs = data_df_split['img']
    masks = data_df_split['mask']

    imgs, masks = crop_resize_data(imgs,masks,new_img_size)
    
    data_df_split = data_df_split.drop('img',axis=1).drop('mask',axis=1)
    
    batch_save_name = "data_df_split_"+str(format(index, "04"))
    
    data_df_split.to_pickle(df_path + batch_save_name + "_df.pkl")
    
    np.save(img_path + batch_save_name + "_img.dat",imgs)
    np.save(mask_path + batch_save_name + "_mask.dat",masks)
    
    
def construct_data_batches(data_df, df_path, img_path, mask_path, new_img_size):

    create_folder(df_path)
    create_folder(img_path)
    create_folder(mask_path)
    
    nr_splits = max(1,int(float(len(data_df)) / NR_ENTRIES_PER_SPLIT) + 1)
    
    print("Nr splits = {}".format(nr_splits))
    
    data_df_splits = np.array_split(data_df, nr_splits)
    
    df_path_list = [df_path] * nr_splits
    img_path_list = [img_path] * nr_splits
    mask_path_list = [mask_path] * nr_splits
    indexes = range(nr_splits)
    new_img_size_list = [new_img_size] * nr_splits
    
    all_data = zip(data_df_splits, df_path_list, img_path_list, mask_path_list, indexes, new_img_size_list)
    
    for data_df_split, df_path, img_path, mask_path, index, new_img_size in tqdm(all_data):
        df = construct_batch((data_df_split, df_path, img_path, mask_path, index, new_img_size))

#     pool = Pool(8)
#     pool.map(construct_batch,all_data)



def merge_df_with_data(df_path,img_path,mask_path):
    
    batch_df = pd.read_pickle(df_path)    

    imgs = np.load(img_path)
    masks = np.load(mask_path)
    
    batch_df.loc[:,'img'] = [img for img in imgs]
    batch_df.loc[:,'mask'] = [mask for mask in masks]
    
    return batch_df


def read_data_batches_in_df(df_path, img_path, mask_path, nr_batches = None):
    
    df_batches_paths = sorted([df_path + path for path in os.listdir(df_path)])
    img_batches_paths = sorted([img_path + path for path in os.listdir(img_path)])
    mask_batches_paths = sorted([mask_path + path for path in os.listdir(mask_path)])
        
    if(nr_batches == None):
        nr_batches = len(df_batches_paths)
        
    df_batches_paths = df_batches_paths[:nr_batches]
    img_batches_paths = img_batches_paths[:nr_batches]
    mask_batches_paths = mask_batches_paths[:nr_batches]
    
    return pd.concat([merge_df_with_data(df_path, img_path , mask_path) \
                               for df_path, img_path, mask_path in tqdm(zip(df_batches_paths,\
                                                                            img_batches_paths,\
                                                                            mask_batches_paths ))],axis=0)

def rotate_image_cw(img):
    return rotate_image(img,-15)

def rotate_image_ccw(img):
    return rotate_image(img,15)

def flip_image(img):
    return cv2.flip(img,1)


def get_augment_techniques():
    return [("flip",flip_image),
            ("rotate_cw",rotate_image_cw),
            ("rotate_ccw",rotate_image_ccw)
           ]

def augment_batches(df_path, img_path, mask_path, new_img_size):
    
    batch_paths = sorted([batch_path for batch_path in os.listdir(df_path) if "augm" not in batch_path])
    
    print("Nr batches = {}".format(len(batch_paths)))
    
    for index,batch_path in tqdm(list(enumerate(batch_paths))):
        
        batch_name = batch_path.split(".")[0][:-3]

        batch_data_df = pd.read_pickle(df_path + batch_path)

        batch_imgs = np.load(img_path + batch_name + "_img.dat.npy")
        batch_masks = np.load(mask_path + batch_name + "_mask.dat.npy")
        
        for name,augment_function in get_augment_techniques():
            
            augm_df_batch_name = batch_name+"_"+name+"_df_augm"
            augm_mask_batch_name = batch_name+"_"+name+"_mask_augm"
            augm_img_batch_name = batch_name+"_"+name+"_img_augm"

            augm_img = np.stack([augment_function(img) for img in batch_imgs])
            augm_mask = np.stack([augment_function(mask) for mask in batch_masks])
            augm_mask = np.expand_dims(augm_mask,axis=3)
            
            batch_data_df.to_pickle(df_path+augm_df_batch_name+".pkl") # same df
            np.save(img_path + augm_img_batch_name + ".dat",augm_img)
            np.save(mask_path + augm_mask_batch_name + ".dat",augm_mask)
        
        
def compute_global_mean(img_path):
    
    split_paths = sorted([img_path + split_path for split_path in os.listdir(img_path)])

    batch_lengths = []
    batch_averages = []
    
    for split_path in tqdm(split_paths):

        img_batch = np.load(split_path)
        
        nr_imgs ,img_width, img_height, nr_color_channels = img_batch.shape
        
        img_batch = img_batch.reshape((nr_imgs, img_width*img_height, nr_color_channels))

        avg_per_pic = np.average(img_batch,axis=1)
        batch_avg = np.average(avg_per_pic,axis=0)
        
        batch_lengths.append(len(img_batch))        
        batch_averages.append(batch_avg)

    global_avg = np.average(arr(batch_averages),weights = arr(batch_lengths), axis=0)
    
    return global_avg.astype(np.float32).reshape((3,1,1))

def data_generator(img_path, mask_path):
    
    batch_img_paths = sorted([img_path + batch_path for batch_path in os.listdir(img_path)])
    batch_mask_paths = sorted([mask_path + batch_path for batch_path in os.listdir(mask_path)])
    
    nr_batches = len(batch_img_paths)
        
    while(1):
        
        for batch_img_path, batch_mask_path in zip(batch_img_paths,batch_mask_paths):
                        
            imgs = np.load(batch_img_path)
            masks = np.load(batch_mask_path)
        
            imgs = np.transpose(imgs,(0,3,1,2))
            masks = np.transpose(masks,(0,3,1,2))

            yield(imgs,masks)

def get_pred_df(data_df, model):
    
    img_data = numpify(data_df['img'])
    img_data = np.transpose(img_data,(0,3,1,2))
    
    predictions = model.predict(img_data,verbose=1)
    
    data_df.loc[:,'pred'] = pd.Series([pred for pred in predictions], index = data_df.index)
    
    return data_df
  
    
def make_prediction_on_dataset(df_path, img_path, mask_path, model, nr_batches = None):

    batch_df_paths = sorted([df_path + batch_path for batch_path in os.listdir(df_path)])
    batch_img_paths = sorted([img_path + batch_path for batch_path in os.listdir(img_path)])
    batch_mask_paths = sorted([mask_path + batch_path for batch_path in os.listdir(mask_path)])

    if(nr_batches == None):
        nr_batches = len(batch_df_paths)
    
    pred_df = []
        
    for batch_df_path, batch_img_path, batch_mask_path in tqdm(zip(batch_df_paths, \
                                                                   batch_img_paths, \
                                                                   batch_mask_paths)[:nr_batches]):
        
        data_df_split = merge_df_with_data(batch_df_path,batch_img_path,batch_mask_path)
        
        pred_df_split = get_pred_df(data_df_split, model)
                
        pred_df.append(pred_df_split)
       
    pred_df = pd.concat(pred_df,axis=0)
    
    return pred_df    

# Paths

In [3]:
new_img_size = 448
NR_ENTRIES_PER_SPLIT = 8

roma_dataset_path =  "../datasets/roma-dataset/"
camvid_dataset_path = "../datasets/camvid/"

base_path = "./data/"

train_base_path = base_path + "train/"
test_base_path = base_path + "test/"

train_df_path = train_base_path +"df/"
train_img_path = train_base_path +"img/"
train_mask_path = train_base_path +"mask/"

test_df_path = test_base_path +"df/"
test_img_path = test_base_path +"img/"
test_mask_path = test_base_path +"mask/"

create_folder(train_base_path)
create_folder(test_base_path)

create_folder(train_df_path)
create_folder(train_img_path)
create_folder(train_mask_path)

create_folder(test_df_path)
create_folder(test_img_path)
create_folder(test_mask_path)
    

# Read Data

In [ ]:
roma_df = get_roma_img_mask_paths(roma_dataset_path)
# camvid_df = get_camvid_img_mask_paths(camvid_dataset_path)

In [ ]:
# data_df = shuffle(pd.concat([roma_df,camvid_df]),random_state = 0)
data_df = roma_df
data_df.shape

In [ ]:
data_df.head()

# Train / Test Split

In [ ]:
TRAIN_PERCENTAGE = 0.8

train_df, test_df = split_train_test(data_df, TRAIN_PERCENTAGE)

print(train_df.shape)
print(test_df.shape)


# Construct Batches

In [ ]:
construct_data_batches(train_df, train_df_path, train_img_path, train_mask_path, new_img_size)

In [ ]:
construct_data_batches(test_df, test_df_path, test_img_path, test_mask_path, new_img_size)

# Viz Data

In [ ]:
split_df = read_data_batches_in_df(train_df_path, train_img_path, train_mask_path, nr_batches = 2)

for _,row in tqdm(list(split_df.iterrows())):
    img = row['img']
    mask = np.squeeze(row['mask'])
    
    plt.subplot(1,3,1)
    plt.imshow(img)
    
    plt.subplot(1,3,2)
    plt.imshow(mask)

    plt.subplot(1,3,3)
    plt.imshow(img)
    plt.imshow(mask,alpha=0.5)

    plt.figure(figsize=(10,10))

plt.show()

# Augment Data

In [ ]:
augment_batches(train_df_path, train_img_path, train_mask_path, new_img_size)

# Model 

In [ ]:
global_mean = compute_global_mean(train_img_path)
global_mean

# Training

In [ ]:
def vgg_preprocess(x):
    x = x - global_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def get_unet(img_width, img_height):

    inputs = Input((3, img_width, img_height))
    x = Lambda(vgg_preprocess, output_shape = (3, 448, 448))(inputs)
    
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(x)
    conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(512, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(256, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy')

    return model


In [ ]:
nr_train_data = get_nr_of_examples(train_df_path)
nr_test_data = get_nr_of_examples(test_df_path)

print(nr_train_data)
print(nr_test_data)

In [ ]:
model = get_unet(new_img_size, new_img_size)

In [ ]:
# figure = SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))
# display(figure)

In [ ]:
model.fit_generator(data_generator(train_img_path, train_mask_path),
      samples_per_epoch = nr_train_data ,
      nb_epoch= 3,               
      validation_data = data_generator(test_img_path, test_mask_path),
      nb_val_samples = nr_test_data, 
      callbacks = [#CSVLogger("./training.txt"), \
                   #ModelCheckpoint("./models-new/model_temp.h5", monitor='val_acc', verbose= 1, save_best_only=True, mode='max')
                  ]
      )

In [ ]:
model.save_weights("./1e_batch.h5")

In [ ]:
model.load_weights("./1e_batch.h5")


## Predict

In [ ]:
pred_data_df = make_prediction_on_dataset(test_df_path, test_img_path, test_mask_path, model, nr_batches = None)

In [ ]:
for _, row in tqdm(list(pred_data_df.iterrows())):
    
    img = row['img']
    mask = np.squeeze(row['mask'])
    pred = np.squeeze(row['pred'])
    
    nr_cols = 3

#     plt.subplot(1,nr_cols,1)
#     plt.imshow(img)

#     plt.subplot(1,nr_cols,2)
#     plt.imshow(mask)

#     plt.subplot(1,nr_cols,3)
#     plt.imshow(pred)

    _,thresh_pred = cv2.threshold(pred,0.9,255,cv2.THRESH_BINARY)
    
#     kernel = np.ones((2,2),np.uint8)
#     opening = cv2.morphologyEx(thresh_pred, cv2.MORPH_OPEN, kernel)
    img_canny = auto_canny(img.astype(np.uint8),0)
    pred_canny = auto_canny(pred.astype(np.uint8),0)
    thresh_canny = auto_canny(thresh_pred.astype(np.uint8),0)
    
#     plt.subplot(1,nr_cols,4)
#     plt.imshow(thresh_pred)

    plt.subplot(1,nr_cols,1)
    plt.imshow(img)

    plt.subplot(1,nr_cols,2)
    plt.imshow(img_canny)

    plt.subplot(1,nr_cols,3)
    plt.imshow(thresh_canny)

    plt.figure(figsize=(10,10))

plt.show()


# Hough

In [ ]:
def plot_hough(image,osc_img):
    # Classic straight-line Hough transform
    h, theta, d = hough_line(image)

    # Generating figure 1
    fig, axes = plt.subplots(1, 2, figsize=(7, 7),
                             subplot_kw={'adjustable': 'box-forced'})
    ax = axes.ravel()

    ax[0].imshow(image, cmap=cm.gray)
    ax[0].set_title('Input image')
    ax[0].set_axis_off()

    ax[1].imshow(osc_img, cmap=cm.gray)
    
    peaks = zip(*hough_line_peaks(h, theta, d, threshold=25))
    nr_peaks = 5
    
    if(len(peaks) < nr_peaks):
        return
    
    angle_2_dist_list = []
    
    
    for _, angle, dist in peaks[:nr_peaks]:
        if(angle < 1.5 and angle > -1.5):
            angle_2_dist_list.append((angle,dist))
        
    angle_2_dist_list = sorted(angle_2_dist_list, key=lambda tup: tup[0])

    selected_angle_dist_list = [angle_2_dist_list[0],angle_2_dist_list[len(angle_2_dist_list)-1]]

    
    lines = []

    for angle,dist in selected_angle_dist_list:
                
        y0 = (dist - 0 * np.cos(angle)) / np.sin(angle)
        y1 = (dist - image.shape[1] * np.cos(angle)) / np.sin(angle)
        
        ax[1].plot((0, image.shape[1]), (y0, y1), '-r')
    
        lines.append(([0,y0],[image.shape[1],y1]))
        
        
    L1 = line(lines[0][0],lines[0][1])
    L2 = line(lines[1][0],lines[1][1])
        
    R = intersection(L1, L2)
    
    r_width, r_height = R
    r_height = max(min(r_height,448),0)
    r_width = max(min(r_width,448),0)
    R = (r_width,r_height)

    if(not R):
        R = (224,0)

    circ = Circle(R,20)
    ax[1].add_patch(circ)

    
    ax[1].set_xlim((0, image.shape[1]))
    ax[1].set_ylim((image.shape[0], 0))
    ax[1].set_axis_off()
    ax[1].set_title('Detected lines ')

    plt.tight_layout()

    
    plt.show()
    
#     return R


In [ ]:
for _, row in tqdm(list(pred_data_df.iterrows())):
    
    img = row['img']
    pred = np.squeeze(row['pred'])

    _,thresh_pred = cv2.threshold(pred,0.9,255,cv2.THRESH_BINARY)
    canny = auto_canny(thresh_pred.astype(np.uint8),0)
    
    plot_hough(canny,img)

# OSC 

In [ ]:
osc_path = "../datasets/validated-imgs/lane-4/"
osc_img_paths = [osc_path + path for path in sorted(os.listdir(osc_path))]
len(osc_img_paths)

In [ ]:
osc_imgs = np.stack([arr(PIL.Image.open(path)) for path in tqdm(osc_img_paths)])

In [ ]:
for index in range(10):
    plt.imshow(osc_imgs[index])
    plt.figure()
plt.show()

In [ ]:
osc_imgs_format = np.transpose(normalize_imgs(osc_imgs),(0,3,1,2))

In [ ]:
osc_preds = model.predict(osc_imgs_format,verbose = 1, batch_size = 1)
osc_preds = np.squeeze(osc_preds)
osc_preds.shape

In [ ]:
for osc_img, osc_pred in tqdm(zip(osc_imgs,osc_preds)[:100]):
    
    _,thresh_pred = cv2.threshold(osc_pred,0.1,255,cv2.THRESH_BINARY)
    
    plt.subplot(1,3,1)
    plt.imshow(osc_img)

    plt.subplot(1,3,2)
    plt.imshow(thresh_pred)

    plt.subplot(1,3,3)
    plt.imshow(osc_img)
    plt.imshow(thresh_pred,alpha=0.5)

    plt.figure(figsize=(10,10))
plt.show()

In [ ]:
for img,mask in tqdm(zip(imgs,masks)[:100]):
        
    nr_cols = 3
    sq_mask = np.squeeze(mask)
    
    plt.subplot(1,nr_cols,1)
    plt.imshow(img)

    plt.subplot(1,nr_cols,2)
    plt.imshow(sq_mask)

    plt.subplot(1,nr_cols,3)
    plt.imshow(img)
    plt.imshow(sq_mask,alpha=0.5)

    plt.figure(figsize=(10,10))
plt.show()


In [ ]:
# def plot_slices_merged(plot_images,plot_masks, predictions, nr_slices_in_pic, total_no_images = None):
    
#     if(total_no_images == None):
#         total_no_images = len(plot_images)
    
#     nr_cols = 3
    
#     step = int(math.sqrt(nr_slices_in_pic))
    
#     for i in range(0,total_no_images,nr_slices_in_pic):
#         image_slices = plot_images[i:i+nr_slices_in_pic,...]
#         mask_slices = plot_masks[i:i+nr_slices_in_pic,...]
#         prediction_slices = predictions[i:i+nr_slices_in_pic,...]
        
#         merged_images = merge_slices(image_slices, step)
#         merged_masks = merge_slices(mask_slices, step)
#         merged_predictions = merge_slices(prediction_slices, step)
        
#         plt.subplot(1,nr_cols,1)
#         plt.imshow(merged_images)
        
#         plt.subplot(1,nr_cols,2)
#         plt.imshow(np.squeeze(merged_masks))

#         plt.subplot(1,nr_cols,3)
#         plt.imshow(np.squeeze(merged_predictions))

#         plt.figure(figsize=(10,10))
        
#     plt.show()
    
# def merge_slices(image_slices,step):
#     cols = []
#     number_bboxes = len(image_slices)
    
#     for i in range(0,number_bboxes,step):
#         current_col = image_slices[i:i+step,...]
#         current_col = np.concatenate(current_col,axis=0)
#         cols.append(current_col)
#     return np.concatenate(cols,axis=1)



# def construct_bboxes(image_size, slice_size):
                    
#     imageWidth = imageHeight = image_size
#     sliceHeight = sliceWidth = slice_size
    
#     left = 0 # Set the left-most edge
#     upper = 0 # Set the top-most edge

#     image_bboxes = []
#     mask_bboxes = []
    
#     while (left < imageWidth):
#         while (upper < imageHeight):
#             # If the bottom and right of the cropping box overruns the image.
#             if (upper + sliceHeight > imageHeight and \
#                 left + sliceWidth > imageWidth):
#                 bbox = (left, upper, imageWidth, imageHeight)
#             # If the right of the cropping box overruns the image
#             elif (left + sliceWidth > imageWidth):
#                 bbox = (left, upper, imageWidth, upper + sliceHeight)
#             # If the bottom of the cropping box overruns the image
#             elif (upper + sliceHeight > imageHeight):
#                 bbox = (left, upper, left + sliceWidth, imageHeight)
#             # If the entire cropping box is inside the image,
#             # proceed normally.
#             else:
#                 bbox = (left, upper, left + sliceWidth, upper + sliceHeight)
               
#             image_bboxes.append(bbox)
            
#             upper += sliceHeight
#         left += sliceWidth # Increment the vertical position
#         upper = 0

#     return image_bboxes



def slice_imgs(imgs, masks, bboxes):
        
    total = len(imgs) * len (bboxes)
    
    imgs_slices = []
    mask_slices = []

    for index in tqdm(range(len(imgs))):

        raw_img = imgs[index]
        raw_mask = masks[index]
            
        for bbox in bboxes:
            
            img = raw_img.crop(bbox)
            mask = raw_mask.crop(bbox)
            
            imgs_slices.append(img)
            mask_slices.append(mask)
            
    imgs_slices = np.stack(imgs_slices)
    mask_slices = np.stack(mask_slices)

    return imgs_slices, mask_slices

    


In [ ]:
data = np.random.rand(50,448,448,3)
data.shape

In [ ]:
mean = np.mean(data)  # mean for data centering
mean

In [ ]:
result = sum(np.ravel((data - mean)**2))
result

In [ ]:
data_1 = np.random.rand(15,448,448,3)
data_1.shape

In [ ]:
mean_1 = np.mean(data_1)
mean_1

In [ ]:
result_1 = 
result_1sum(np.ravel((data_1 - mean_1)**2))

In [ ]:
total_res = math.sqrt((result + result_1)/(65*448*448*3))
total_res

In [ ]:
total_data = np.concatenate([data,data_1])
total_data.shape

In [ ]:
np.std(total_data)

In [ ]:
print(data_1.shape)
print(np.ravel(data_1).shape)


In [ ]:
batches = [data,data_1]

partial_results = []
total_nr_examples = 0.

for batch in batches:
    total_nr_examples += len(np.ravel(batch))
    mean = np.mean(batch)
    partial_result = sum(np.ravel((batch - mean)**2))
    partial_results.append(partial_result)
    
total_result = sum(partial_results)
stddev =  math.sqrt( total_result / total_nr_examples)
print(stddev)